In [14]:
import spacy

In [6]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import json
from nltk.stem.snowball import SnowballStemmer
import os
import re
import requests

In [1]:
!pip install bertopic --quiet

ERROR: hdbscan 0.8.27 has requirement joblib>=1.0, but you'll have joblib 0.14.1 which is incompatible.
ERROR: sentence-transformers 0.4.1.2 has requirement torch>=1.6.0, but you'll have torch 1.5.0+cpu which is incompatible.
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
mf_df = pd.read_csv('foundations_count.csv')

In [4]:
mf_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,GKGRECORDID,DATE,DocumentIdentifier,V2Tone,title,authors,publish_date,...,text_tokens,domain,tt_v3,text_list,word_count,authority/subversion,care/harm,fairness/cheating,loyalty/betrayal,sanctity/degradation
0,0,0,10667,20190227000000-1555,2019-02-27 00:00:00,https://www.cbsnews.com/news/thousands-of-migr...,"-7.41687979539642,1.79028132992327,9.207161125...",Thousands of migrant children were sexually ab...,"['Camilo Montoya-Galvez', 'Camilo Montoya-Galv...",NaN,...,story update recent statement department healt...,cbsnews,story update recent statement department healt...,"['stori', 'updat', 'recent', 'statement', 'dep...",514,12.0,33.0,11.0,7.0,4.0
1,1,1,10676,20190227000000-318,2019-02-27 00:00:00,https://www.huffpost.com/entry/california-salm...,"-0.0779220779220777,2.7012987012987,2.77922077...",This Project Aimed To Be A Model For Saving Fa...,['Margiana Petersen-Rockney'],2019-02-26 15:22:35-05:00,...,story produce originally publish grist reprodu...,huffpost,story produce originally publish grist reprodu...,"['stori', 'produc', 'origin', 'publish', 'gris...",1848,21.0,31.0,18.0,50.0,7.0
2,2,2,10675,20190227000000-315,2019-02-27 00:00:00,https://www.huffpost.com/entry/us-military-int...,"-3.91414141414141,3.15656565656566,7.070707070...",The U.S. Is Moving Closer To Military Interven...,['On Assignment For Huffpost'],2019-02-26 17:07:51-05:00,...,cúcuta colombia u.s. move close military inter...,huffpost,cúcuta colombia u.s. move close military inter...,"['cúcuta', 'colombia', 'u.s.', 'move', 'close'...",771,28.0,32.0,4.0,19.0,1.0
3,3,3,10674,20190227000000-181,2019-02-27 00:00:00,http://www.latimes.com/socal/burbank-leader/ne...,"6.957928802589,7.92880258899676,0.970873786407...",No plateau in sight for Burroughs High senior ...,"['Staff Writer', 'Andrew J. Campa Covers The E...",2019-02-26 00:00:00,...,glisten tiara royal purple line sash burroughs...,latimes,glisten tiara royal purple line sash burroughs...,"['glisten', 'tiara', 'royal', 'purpl', 'line',...",265,5.0,0.0,4.0,5.0,5.0
4,4,4,10673,20190227000000-2604,2019-02-27 00:00:00,https://uk.reuters.com/article/us-usa-congress...,"0.770416024653313,3.54391371340524,2.773497688...",U.S. House Democrats introduce sweeping 'Medic...,['Yasmeen Abutaleb'],2019-02-26 22:18:42+00:00,...,washington reuters democrats u.s. house repres...,reuters,washington reuters democrats u.s. house repres...,"['washington', 'reuter', 'democrat', 'u.s.', '...",314,9.0,1.0,0.0,3.0,35.0


In [7]:
#loading moral foundations dictionary
data = requests.get('https://raw.githubusercontent.com/ds-modules/XRHETOR-R1A/v1.0/mft_data/haidt_dict.json')

mft = data.json()

mft.keys()

dict_keys(['authority/subversion', 'care/harm', 'fairness/cheating', 'loyalty/betrayal', 'sanctity/degradation'])

In [8]:
#calculating percent of moral words for each record
for foundation in mft.keys():
    mf_df[foundation] = (mf_df[foundation]/mf_df['word_count'])

In [10]:
#getting rid of publishers will low number of articles
mf_df = mf_df.loc[~mf_df['domain'].isin(['buzzfeed', 'msn', 'cnbc', 'knpr', 'wnpr', 'sabcnews'])]

In [11]:
mf_df.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,GKGRECORDID,DATE,DocumentIdentifier,V2Tone,title,authors,publish_date,...,text_tokens,domain,tt_v3,text_list,word_count,authority/subversion,care/harm,fairness/cheating,loyalty/betrayal,sanctity/degradation
0,0,0,10667,20190227000000-1555,2019-02-27 00:00:00,https://www.cbsnews.com/news/thousands-of-migr...,"-7.41687979539642,1.79028132992327,9.207161125...",Thousands of migrant children were sexually ab...,"['Camilo Montoya-Galvez', 'Camilo Montoya-Galv...",NaN,...,story update recent statement department healt...,cbsnews,story update recent statement department healt...,"['stori', 'updat', 'recent', 'statement', 'dep...",514,0.023346,0.064202,0.021401,0.013619,0.007782
1,1,1,10676,20190227000000-318,2019-02-27 00:00:00,https://www.huffpost.com/entry/california-salm...,"-0.0779220779220777,2.7012987012987,2.77922077...",This Project Aimed To Be A Model For Saving Fa...,['Margiana Petersen-Rockney'],2019-02-26 15:22:35-05:00,...,story produce originally publish grist reprodu...,huffpost,story produce originally publish grist reprodu...,"['stori', 'produc', 'origin', 'publish', 'gris...",1848,0.011364,0.016775,0.009740,0.027056,0.003788
2,2,2,10675,20190227000000-315,2019-02-27 00:00:00,https://www.huffpost.com/entry/us-military-int...,"-3.91414141414141,3.15656565656566,7.070707070...",The U.S. Is Moving Closer To Military Interven...,['On Assignment For Huffpost'],2019-02-26 17:07:51-05:00,...,cúcuta colombia u.s. move close military inter...,huffpost,cúcuta colombia u.s. move close military inter...,"['cúcuta', 'colombia', 'u.s.', 'move', 'close'...",771,0.036316,0.041505,0.005188,0.024643,0.001297
3,3,3,10674,20190227000000-181,2019-02-27 00:00:00,http://www.latimes.com/socal/burbank-leader/ne...,"6.957928802589,7.92880258899676,0.970873786407...",No plateau in sight for Burroughs High senior ...,"['Staff Writer', 'Andrew J. Campa Covers The E...",2019-02-26 00:00:00,...,glisten tiara royal purple line sash burroughs...,latimes,glisten tiara royal purple line sash burroughs...,"['glisten', 'tiara', 'royal', 'purpl', 'line',...",265,0.018868,0.000000,0.015094,0.018868,0.018868
4,4,4,10673,20190227000000-2604,2019-02-27 00:00:00,https://uk.reuters.com/article/us-usa-congress...,"0.770416024653313,3.54391371340524,2.773497688...",U.S. House Democrats introduce sweeping 'Medic...,['Yasmeen Abutaleb'],2019-02-26 22:18:42+00:00,...,washington reuters democrats u.s. house repres...,reuters,washington reuters democrats u.s. house repres...,"['washington', 'reuter', 'democrat', 'u.s.', '...",314,0.028662,0.003185,0.000000,0.009554,0.111465


In [12]:
#dropping unneeded columns
mf_df_abridged.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1', ], axis=1, inplace=True)

In [13]:
mf_df_abridged['domain'].value_counts()

reuters           23797
foxnews           21933
nytimes           17927
usatoday          14400
cnn               13880
latimes           12872
breitbart         12464
nbcnews            9298
cbsnews            7096
npr                5251
nationalreview     4179
vox                3463
msnbc              1153
huffpost            970
Name: domain, dtype: int64

In [18]:
author_list = set()
for idx, authors in enumerate(list(mf_df['authors'])):

    authors = authors.strip('[').strip(']').replace("'", "" ).replace(",", "").replace(".", "")
    authors = authors.split(' ')
    author_list.update(authors)



#     for name in authors:

#         if name not in author_list:
#             author_list.append(name.lower())

In [22]:
author_list.remove('')

In [27]:
cls = spacy.util.get_lang_class('en')


cls.Defaults.stop_words.add('updat')

cls.Defaults.stop_words.add('stori')

for word in ['recent', 'co', 'statement']:
    cls.Defaults.stop_words.add(word)

for word in author_list:
    cls.Defaults.stop_words.add(word)

nlp = spacy.load('en', disable=['tagger', 'parser', 'ner'])

In [28]:
def spacy_stop_remove(text):
    text =text.strip('[').strip(']').replace("'", "" ).replace(",", "").replace(".", "")
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop]
    text_str = (" ").join(tokens)
    return tokens

In [ ]:
mf_df['text_stop_removed'] = mf_df['tt_v3'].apply(lambda x: spacy_stop_remove(x))

In [45]:
mf_df['text_stop_removed'] = mf_df['text_stop_removed'].str.join(' ')

In [80]:
mf_df.to_csv('domain_stop_removed_bias_text.csv')